# C1 W4 Group 8 - Task 3


In [ ]:
from PIL import Image

from src.data import AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST
from src.paths import BBDD_PATH, QSD1_W4_PATH, QSD1_NON_AUGMENTED_W4_PATH, WEEK_4_RESULTS_PATH
from tqdm import tqdm

In [ ]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QSD1_W4_PATH_LIST = sorted(QSD1_W4_PATH.glob("*.jpg"))
QSD1_NON_AUGMENTED_W4_PATH_LIST = sorted(QSD1_NON_AUGMENTED_W4_PATH.glob("*.jpg"))

In [ ]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [ ]:
query_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [ ]:
non_augmented_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QSD1_NON_AUGMENTED_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(non_augmented_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [ ]:
for file, augmentation, frames, gt in zip(QSD1_W4_PATH_LIST, AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST):
    print(f"File={file.stem}, Aug={augmentation}, Frames={frames}, GT={gt}")

## Background Removal

In [ ]:
from src.background import get_paintings_cropped_images, resize_image

In [ ]:
IMAGE_SIZE = 2**9  # IMPORTANT: we define a same size and apply it to detect the images and also for retrieval


In [ ]:
cropped_query_image_list_d1 = []
pbar = tqdm(zip(QSD1_W4_PATH_LIST, query_d1_image_PIL_list))
for name, image in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    image_list = get_paintings_cropped_images(image, image_size=IMAGE_SIZE, output_size=IMAGE_SIZE)
    cropped_query_image_list_d1.append(image_list)

In [ ]:
database_image_PIL_list = [Image.fromarray(resize_image(db_image, IMAGE_SIZE)) for db_image in database_image_PIL_list]

## Image Denoising

In [ ]:
from src.denoising import denoise_image

In [ ]:
denoised_query_d1_PIL_list = []
for image_tuple in cropped_query_image_list_d1:
    temp = []
    for img in image_tuple:
        temp.append(denoise_image(img))
    denoised_query_d1_PIL_list.append(temp)

## Retrieval

In [ ]:
from src.descriptors import ImageRetrievalSystem, SIFTDescriptor, HOGDescriptor, ORBDescriptor

In [ ]:
descriptors = {
    'SIFT': SIFTDescriptor(max_features=500),
    'HOG13': HOGDescriptor(pixels_per_cell=12, cells_per_block=4),
    'ORB': ORBDescriptor(),
}
descriptor_path = WEEK_4_RESULTS_PATH / 'Task_1' / 'descriptors'  

retrieval_system = ImageRetrievalSystem(descriptors, descriptor_path, log=False)

In [ ]:
#Set number of top similar images to retrieve
K = 10

results = {}
g_0 = {}

descriptor_name = "HOG13"

results[descriptor_name] = retrieval_system.retrieve_similar_images(descriptor_name, database_image_PIL_list, denoised_query_d1_PIL_list, None, K)

## Evaluation

In [ ]:
from src.metrics import MeanAveragePrecisionAtK

map = MeanAveragePrecisionAtK()

In [ ]:
GT_mod = []
for element in GT_QSD1_W4_LIST:
    if len(element) == 2:
        GT_mod.append([element[0]])
        GT_mod.append([element[1]])
    else:
        GT_mod.append([element[0]])

retrieved_images_d1 = results[descriptor_name]
results_mod = []
for element in retrieved_images_d1:
    for e in element:
        results_mod.append(e)

print(GT_mod)
print(results_mod)

In [ ]:
result = map.compute(GT_mod, results_mod, k=10)
print(f"MAP QSD1_W4 Method Week 3: {result}")